In [18]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from azapi import AZlyrics
import time
import random
import requests
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import subprocess
import re



client_id = 'c0c9215f552446099320224027317acb'  
client_secret = '53be15627b244404a0786d64a1f94049'  

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



In [77]:

ua = UserAgent()

def extract_lyrics_with_selenium(url):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(url)

    
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.lyr_data div._inner"))
        )
        lyrics_html = element.get_attribute('innerHTML')
        print("lyrics from gaana")
    except Exception as e:
        lyrics = f"Failed to find the lyrics due to: {str(e)}"
        print("switching to azlyrics")
        lyrics = get_lyrics_from_jiosaavn(jiosaavn_link)
        print("lyrics from jiosaavn")
        
        
    else:
        soup = BeautifulSoup(lyrics_html, 'html.parser')
        lyrics = soup.get_text(separator=' ', strip=True)
    finally:
        driver.quit()

    return lyrics

def get_duckduckgo_url(song_name, artist_name):
    query = f"{song_name} {artist_name} lyrics site:gaana.com"
    headers = {
        'User-Agent': ua.random
    }
    
    # Use a fallback User-Agent if needed
    try:
        response = requests.get(f"https://duckduckgo.com/html/?q={query}", headers=headers)
    except requests.exceptions.RequestException as e:
        print(f"RequestException: {e}")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        response = requests.get(f"https://duckduckgo.com/html/?q={query}", headers=headers)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('a', class_='result__a')
    
    for result in results:
        href = result.get('href')
        parsed_url = urlparse(href)
        
        if not parsed_url.scheme:
            href = urljoin("https://duckduckgo.com", href)
        
        if "gaana.com" in href:
            print(f"Found Gaana URL: {href}")  # Debug print
            return href
        
    return None

def AZ(song_title,artist_name_l):
    api = AZlyrics("duckduckgo",accuracy=0.4)
    api.title = song_title
    api.artist = artist_name_l
    lyrics=api.getLyrics()
    time.sleep(random.uniform(5, 10))
    if lyrics in ["Duckduckgo found nothing!", "Similarity < 0.4",0]:

        api = AZlyrics("duckduckgo",accuracy=0.4)
        api.title = song_title
        lyrics = api.getLyrics()
        time.sleep(random.uniform(5, 10))
    return lyrics


def download_track(track_url):
    command = ["spotdl", "download", track_url]
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Download Successful")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error during download")
        print(e.stderr)

def clean_name(name):
    # Remove any text after a hyphen or within brackets
    name = re.sub(r" - .*", "", name)  # Removes anything after " - " including the hyphen
    name = re.sub(r"\(.*?\)", "", name)  # Removes anything inside parentheses
    name = re.sub(r"\[.*?\]", "", name)  # Removes anything inside square brackets
    name = re.sub(r"\{.*?\}", "", name)  
    return name.strip()

def search_song_jiosaavn(song_name):
    url = "http://192.168.1.156:5100/result/"  # Adjust the endpoint accordingly
    params = {'query': song_name}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data and len(data) > 0:
            return data[0]['perma_url']  # Adjust based on actual API response structure
    return None

# Function to fetch lyrics from JioSaavn link
def get_lyrics_from_jiosaavn(song_link):
    url = "http://192.168.1.156:5100/lyrics/"  # Adjust the endpoint accordingly
    params = {'query': song_link, 'lyrics': 'true'}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data.get('lyrics')  # Adjust based on actual API response structure
    return "Lyrics not found."


In [79]:
def get_playlist_details(playlist_id):

    # Check if the CSV already exists to determine if we need to include headers
    try:
        pd.read_csv('1.csv')
        add_header = False
    except FileNotFoundError:
        add_header = True

    # Fetch the playlist tracks
    results = sp.playlist_tracks(playlist_id, limit=15)
    track_data = [      ]

    # Loop through each track and gather data
    for item in results['items']:
        track = item['track']
        track_id = track['id']
        track_name = track['name']
        track_popularity = track['popularity']
        track_duration = track['duration_ms']
        album_name = track['album']['name']
        album_type = track['album']['album_type']
        release_date = track['album']['release_date']
        artist_name = track['artists'][0]['name']
        artist_id = track['artists'][0]['id']
        cleaned_track_name = clean_name(track_name)
        print(cleaned_track_name)
        print(artist_name)

        # Replace the URL with your track's Spotify URL
        track_url = "https://open.spotify.com/track/"+track_id
        download_track(track_url)

        global song_title, artist_name_l

        song_title = cleaned_track_name  # Replace with the actual song title
        artist_name_l= artist_name

        global jiosaavn_link

        jiosaavn_link = search_song_jiosaavn(cleaned_track_name)
        
        
        url = get_duckduckgo_url(song_title, artist_name_l)
        if url:
            print(f"Found URL: {url}")
            lyrics = extract_lyrics_with_selenium(url)
            time.sleep(random.uniform(5, 10))
            if lyrics is None or not lyrics.strip() or lyrics == '0' or lyrics==0:  # This checks if lyrics are empty or just whitespace
                print("Lyrics not found, switching to alternative method.")
                lyrics = AZ(song_title, artist_name_l)
                time.sleep(random.uniform(5, 10))
        else:
            print("No suitable URL found.")
            lyrics=AZ(song_title,artist_name_l)
            time.sleep(random.uniform(5, 10))
        
        # Fetch artist's genres (used as the album's genres)
        artist_info = sp.artist(artist_id)
        artist_genres = artist_info['genres']

        # Fetch audio features
        audio_features = sp.audio_features(track_id)[0]

        # Collecting all the needed info in a dictionary
        track_info = {
            'Playlist ID': playlist_id,
            'Track ID': track_id,
            'Track Name': track_name,
            'Track Popularity': track_popularity,
            'Track Duration (ms)': track_duration,
            'Artist Name': artist_name,
            'Artist Genres': artist_genres,
            'Album Name': album_name,
            'Album Genres': artist_genres,
            'Album Type': album_type,
            'Release Date': release_date,
            'Danceability': audio_features['danceability'],
            'Energy': audio_features['energy'],
            'Key': audio_features['key'],
            'Loudness': audio_features['loudness'],
            'Mode': audio_features['mode'],
            'Speechiness': audio_features['speechiness'],
            'Acousticness': audio_features['acousticness'],
            'Instrumentalness': audio_features['instrumentalness'],
            'Liveness': audio_features['liveness'],
            'Valence': audio_features['valence'],
            'Tempo': audio_features['tempo'],
            'Time Signature': audio_features['time_signature'],
            'Lyrics': lyrics 
        }

        # Append each track's info to the list
        track_data.append(track_info)

    # Create a DataFrame
    df = pd.DataFrame(track_data)

    # Append or create the CSV file
    df.to_csv('1.csv', mode='a', header=add_header, index=False)
    print(f"Data for playlist {playlist_id} appended to 'playlist_tracks_info.csv'.")

# Example usage for multiple playlists
playlist_ids = ["37i9dQZF1DX8xfQRRX1PDm","37i9dQZF1DX14CbVHtvHRB","34KCFSmH01yeaRo9JgEUFl","5QOrHPIzTFh80WhHmbOcCp"]
for pid in playlist_ids:
    get_playlist_details(pid)


What Jhumka ?
Pritam
Found Gaana URL: https://gaana.com/lyrics/what-jhumka-from-rocky-aur-rani-kii-prem-kahaani-1
Found URL: https://gaana.com/lyrics/what-jhumka-from-rocky-aur-rani-kii-prem-kahaani-1
lyrics from gaana
Bom Diggy Diggy
Zack Knight
Found Gaana URL: https://gaana.com/lyrics/bom-diggy-diggy
Found URL: https://gaana.com/lyrics/bom-diggy-diggy
lyrics from gaana
Teri Baaton Mein Aisa Uljha Jiya Title Song
Raghav
Found Gaana URL: https://gaana.com/lyrics/teri-baaton-mein-aisa-uljha-jiya-title-song
Found URL: https://gaana.com/lyrics/teri-baaton-mein-aisa-uljha-jiya-title-song
lyrics from gaana
Thumkeshwari
Sachin-Jigar
Found Gaana URL: https://gaana.com/lyrics/thumkeshwari-1
Found URL: https://gaana.com/lyrics/thumkeshwari-1
lyrics from gaana
Angaaron
Shreya Ghoshal
Found Gaana URL: https://duckduckgo.com/l/?uddg=https%3A%2F%2Fgaana.com%2Flyrics%2Fangaaron%2Dfrom%2Dpushpa%2D2%2Dthe%2Drule&rut=e765c3b98be3a5b7270acb0c6cae9e16e873ceca39a07e2085bdd7b47ca84295
Found URL: https://d

In [21]:
import requests
import pandas as pd
import json
from tqdm import tqdm
import time

def get_spotify_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(url, headers=headers, data=payload, auth=(client_id, client_secret))
    token = response.json().get('access_token')
    return token

def safe_request(url, headers):
    max_retries = 5  # Set a maximum number of retries
    retry_delay = 1  # Start with a 1 second delay
    for attempt in range(max_retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 429:  # Rate limit exceeded
            retry_after = int(response.headers.get('Retry-After', retry_delay))
            print(f"Rate limit exceeded. Retrying after {retry_after} seconds.")
            if retry_after > 3600:  # If the wait time is longer than an hour
                print(f"Long wait detected. Pausing and resuming after {retry_after} seconds.")
            time.sleep(retry_after)
            retry_delay *= 2  # Double the delay for the next attempt if needed again
        elif response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()
    raise Exception("Max retries exceeded for API request")

def get_artist_details(artist_id, token):
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {'Authorization': f'Bearer {token}'}
    try:
        artist_data = safe_request(url, headers)
        return {'artist_genre': ', '.join(artist_data['genres']) if 'genres' in artist_data and artist_data['genres'] else 'No Genre'}
    except Exception as e:
        print(f"Error fetching artist details: {e}")
        return {'artist_genre': 'No Genre'}  # Return a default value in case of error

def get_track_details(track_id, token):
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = {'Authorization': f'Bearer {token}'}
    try:
        track_data = safe_request(url, headers)
        artist_info = get_artist_details(track_data['artists'][0]['id'], token)
        track_info = {
            'song_title': track_data['name'],
            'song_id': track_data['id'],
            'track_popularity': track_data['popularity'],
            'track_duration': track_data['duration_ms'],
            'explicit': track_data['explicit'],
            'artist_name': track_data['artists'][0]['name'],
            'artist_id': track_data['artists'][0]['id'],
            'artist_genre': artist_info['artist_genre'],
            'album_id': track_data['album']['id'],
            'album_type': track_data['album']['album_type'],
            'release_date': track_data['album']['release_date']
        }
        return track_info
    except Exception as e:
        print(f"Error fetching track details: {e}")
        return {}  # Return an empty dictionary if there's an error

def get_audio_features(track_id, token):
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = {'Authorization': f'Bearer {token}'}
    try:
        audio_features = safe_request(url, headers)
        return audio_features
    except Exception as e:
        print(f"Error fetching audio features: {e}")
        return {}  # Return an empty dictionary if there's an error

client_id = '0bb409de6deb48be93d4ad4973250c13'  
client_secret = '9c2488e0f3c74bae9e1219d25f0be5b5'  
token = get_spotify_token(client_id, client_secret)

# Load track IDs from JSON file
with open('/Users/sakshamjain/Desktop/SPOTIFY 2.0/songs.json') as file:
    track_ids = json.load(file)

# Process tracks in batches of 500
for i in range(24500, len(track_ids), 500):  # Adjusted from 2000 to 500
    batch = track_ids[i:i+500]
    batch_data = []
    print(f"Starting batch {i//500 + 1}...")
    for index, tid in enumerate(tqdm(batch, desc=f"Fetching batch {i//500 + 1}")):
        track_details = get_track_details(tid, token)
        audio_features = get_audio_features(tid, token)
        if track_details and audio_features:
            combined_data = {**track_details, **audio_features}
            batch_data.append(combined_data)
        else:
            print(f"Warning: Missing data for track ID {tid}")

        if (index + 1) % 20 == 0:  # After every 20 songs, add a delay
            time.sleep(1)  # Sleep for 1 second (adjust based on your needs)

    if batch_data:
        df = pd.DataFrame(batch_data)
        csv_filename = f'spotify_data_{i//500 + 1}.csv'
        df.to_csv(csv_filename, index=False)
        print(f"Batch {i//500 + 1} complete and saved to {csv_filename}.")
    else:
        print(f"No data to save for batch {i//500 + 1}.")

Starting batch 50...


Fetching batch 50:  95%|█████████▌| 475/499 [04:56<00:13,  1.81it/s]

Rate limit exceeded. Retrying after 1 seconds.


Fetching batch 50: 100%|██████████| 499/499 [05:11<00:00,  1.60it/s]

Batch 50 complete and saved to spotify_data_50.csv.
